# mwb_flow.prep Package Example
This notebook provides a complete example of acquiring meterology data and combining it with stream gage data to produce a formatted input data file for running the monthly water balance flow (mwb_flow) model (Gray and McCabe, 2010).
A subset of dates were used (1/1/2016 - 12/31/2016) to reduce the time spent getting gridmet data while testing.

First import necessary packages/modules

In [1]:
import os
from pathlib import Path
import geopandas as gpd
import xarray as xr



mwb_flow_dir = r'C:\Users\CND905\Downloaded_Programs\mwb_flow'
os.chdir(mwb_flow_dir)

from prep.datafile import CreateInputFile
from prep.metdata import get_gridmet_at_points
from prep.datafile import check_format

Initializing mwb_flow.prep module.


Import a shape file with deliniated watershed polygon. This shape file has an attribute table with a column used to index the geometries. In this case, a column gage station numbers was used since the numbers will later be used to merge meterologic data with streamflow data.

In [2]:
exres_pth = Path(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\Examples\data\Lolo_WB_Model_Calibration_Catchments_32611.shp')
exres = gpd.read_file(exres_pth)
# This file is in crs 32611 (WGS84 UTM zone 11N), need it to be 4326 for getting GridMET.
exres = exres.to_crs(4326)

In [3]:
# This example provides a unique identifier column rather than using the default DataFrame index.
# TODO change the gdf_index_col to accept full gage ID numbers?
exres_met = get_gridmet_at_points(exres, 'gageID', start='2016-01-01', end='2016-12-31')

C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\metdata.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords = list(zip(in_geom.geometry.centroid.x, in_geom.geometry.centroid.y))


Retrieving GridMET cells...


C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\metdata.py:68: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gmt_cntrs = gmt_cells.drop_duplicates(subset='cell_id').centroid


70 unique GridMET cells found for 6 input features.
Fetching GridMET data for unique cells...


Cells: 100%|██████████| 70/70 [03:17<00:00,  2.82s/it]


A object with meterology data was made using the CreateInputFile class. This class initialization and methods have been edited from the pydlem.prep module which required lake parameters inputs. This class will be further updated to create a method for merg stream gage data. Note that the data frequency is daily and needs to be reduced to monthly. 
This cell is here to demonstrate/test if the updated CreateInputFile code is working for now.  

In [4]:
test = CreateInputFile(geoms=None, met_data=exres_met, met_source='from_file')
test.data

<xarray.Dataset> Size: 56kB
Dimensions:   (time: 366, location: 6)
Coordinates:
    lat       (location) float64 48B 46.82 46.78 46.75 46.75 46.69 46.69
    lon       (location) float64 48B -114.4 -114.4 -114.2 -114.2 -114.5 -114.3
    elev      (location) float64 48B 1.43e+03 1.237e+03 ... 1.404e+03 1.743e+03
  * location  (location) int64 48B 2800 3000 9600 9500 1000 5500
  * time      (time) datetime64[ns] 3kB 2016-01-01 2016-01-02 ... 2016-12-31
Data variables:
    precip    (time, location) float64 18kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    min_temp  (time, location) float64 18kB 255.8 255.4 256.5 ... 261.5 261.4
    max_temp  (time, location) float64 18kB 265.9 265.1 265.4 ... 269.4 269.4
Attributes:
    featureType:  timeSeries

The daily temperature and precipitation data are summarised to monthly frequencies below. The resample function allows for simple grouping and the application of mathamatical functions. However, doing so changes the dimentions of an array, which makes this tricky. Perhaps a diffrent workflow is needed. 

Here, the daily temperature is averaged to calculate the monthly temperature. 

In [5]:
# Calculate daily mean air temperature then followed by the monthly mean air temperature
mean_temp = ((exres_met.min_temp + exres_met.max_temp) / 2) - 273.15  # also convert to Celcius from GridMET native Kelvin
mean_temp
monthly_temp = mean_temp.resample(time = "MS").mean()

# Convert to a DataArray with attributes and title
Monthly_Temp = xr.DataArray(monthly_temp, coords=monthly_temp.coords, attrs={'standard_name': 'Monthly_Temperature', 'units': 'Celcius'})
Monthly_Temp.name = 'mo_temp'
Monthly_Temp

<xarray.DataArray 'mo_temp' (time: 12, location: 6)> Size: 576B
array([[-2.67580645, -2.73321386, -2.65173697, -2.65211694, -3.10505865,
        -3.32720898],
       [ 1.02364532,  1.06590038,  1.50954907,  1.3231681 ,  0.35258621,
         0.07293853],
       [ 1.9218894 ,  2.05227001,  2.39950372,  2.13024194,  1.12258065,
         0.64424965],
       [ 7.77738095,  7.78234568,  7.99435897,  7.72541667,  6.77962121,
         6.24601449],
       [ 9.32327189,  9.54450418,  9.61761787,  9.30625   ,  8.4691349 ,
         7.76514727],
       [14.385     , 14.50462963, 15.00717949, 14.70458333, 13.29219697,
        13.01797101],
       [15.66036866, 15.76451613, 16.57022333, 16.21139113, 14.54032258,
        14.43134642],
       [15.93294931, 15.89396655, 16.55930521, 16.27308468, 14.73760997,
        14.70680224],
       [10.04880952, 10.27141975, 10.71064103, 10.40052083,  9.23348485,
         8.92398551],
       [ 4.61313364,  4.84832736,  5.22332506,  4.91864919,  4.00293255,
         3.57791024],
       [ 2.56619048,  2.39555556,  2.91897436,  2.77020833,  1.82030303,
         1.72326087],
       [-8.09930876, -7.96833931, -7.76315136, -7.9452621 , -8.22030792,
        -8.64915849]])
Coordinates:
    lat       (location) float64 48B 46.82 46.78 46.75 46.75 46.69 46.69
    lon       (location) float64 48B -114.4 -114.4 -114.2 -114.2 -114.5 -114.3
    elev      (location) float64 48B 1.43e+03 1.237e+03 ... 1.404e+03 1.743e+03
  * location  (location) int64 48B 2800 3000 9600 9500 1000 5500
  * time      (time) datetime64[ns] 96B 2016-01-01 2016-02-01 ... 2016-12-01
Attributes:
    standard_name:  Monthly_Temperature
    units:          Celcius

And the daily precipitation summed to calcualte the montly precipitation. 

In [6]:
# Calculate the monthly precipitation
monthly_precip = exres_met.precip.resample(time = "MS").sum()

# Convert to a DataArray with attributes and title
Monthly_Precip = xr.DataArray(monthly_precip, coords=monthly_precip.coords, attrs={'standard_name': 'Monthy Precipitation', 'units': 'mm'})
Monthly_Precip.name = 'mo_precip'
Monthly_Precip

<xarray.DataArray 'mo_precip' (time: 12, location: 6)> Size: 576B
array([[ 78.75714286,  72.18148148,  65.88461538,  78.68125   ,
        118.68181818, 123.8       ],
       [ 83.54285714,  78.29259259,  66.42307692,  82.46875   ,
        129.82727273, 144.19565217],
       [ 94.82857143,  88.75185185,  82.73846154,  97.33125   ,
        147.70909091, 162.9       ],
       [ 45.35714286,  44.65925926,  50.13846154,  52.54375   ,
         55.74090909,  59.65652174],
       [ 65.68571429,  61.58888889,  72.54615385,  73.1       ,
         69.92272727,  76.00869565],
       [ 25.77142857,  24.25925926,  25.20769231,  26.16875   ,
         27.24090909,  27.55217391],
       [ 55.24285714,  59.52222222,  65.26923077,  64.13125   ,
         64.30909091,  61.97391304],
       [ 10.5       ,  11.32592593,  14.83846154,  14.33125   ,
         10.35909091,  12.19130435],
       [ 57.1       ,  55.27037037,  60.16923077,  63.5125    ,
         66.17727273,  75.30869565],
       [159.71428571, 155.14444444, 166.49230769, 177.55625   ,
        203.30909091, 212.65217391],
       [ 45.85714286,  42.81481481,  35.27692308,  42.8       ,
         68.16363636,  71.03913043],
       [121.        , 115.28518519, 106.27692308, 123.7125    ,
        174.71363636, 188.9173913 ]])
Coordinates:
    lat       (location) float64 48B 46.82 46.78 46.75 46.75 46.69 46.69
    lon       (location) float64 48B -114.4 -114.4 -114.2 -114.2 -114.5 -114.3
    elev      (location) float64 48B 1.43e+03 1.237e+03 ... 1.404e+03 1.743e+03
  * location  (location) int64 48B 2800 3000 9600 9500 1000 5500
  * time      (time) datetime64[ns] 96B 2016-01-01 2016-02-01 ... 2016-12-01
Attributes:
    standard_name:  Monthy Precipitation
    units:          mm

The monthly temperature and precipitation data are merged into a single dataset that can be used as an input for the mwb_flow model. This also includes latitude which is an input as well. 
We can also use the updated check_format function from pydlem and see that the coordinates and required data variables are accounted for. 

In [7]:
metdata_input = xr.merge([Monthly_Temp, Monthly_Precip])
check_format(metdata_input)

All necessary variables exist and are labeled properly.
All necessary coordinates exist and are labeled properly


Lastly, we can view the dataset.

In [8]:
metdata_input

<xarray.Dataset> Size: 1kB
Dimensions:    (location: 6, time: 12)
Coordinates:
    lat        (location) float64 48B 46.82 46.78 46.75 46.75 46.69 46.69
    lon        (location) float64 48B -114.4 -114.4 -114.2 -114.2 -114.5 -114.3
    elev       (location) float64 48B 1.43e+03 1.237e+03 ... 1.404e+03 1.743e+03
  * location   (location) int64 48B 2800 3000 9600 9500 1000 5500
  * time       (time) datetime64[ns] 96B 2016-01-01 2016-02-01 ... 2016-12-01
Data variables:
    mo_temp    (time, location) float64 576B -2.676 -2.733 ... -8.22 -8.649
    mo_precip  (time, location) float64 576B 78.76 72.18 65.88 ... 174.7 188.9
Attributes:
    standard_name:  Monthly_Temperature
    units:          Celcius

A DataArray of the monthly discharge is imported from the mwb_flow.q_datafile modual. This data was summarized by month, labeled with stream gage station number identifiers, and concatinated.

Stream gage data is required for calibration but not as an input into the mwb_flow model so we can keep it seperate. Ideally, the code in mwb_flow.q_datafile will be updated to include code from the MTDNRdata package which is where discharge can be quarried and pulled from. Perhaps the current code can be a 'from file' option for bringing in calibration data.

In [ ]:
Monthly_Q = xr.load_dataarray(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\q_datafile_output.nc')
Monthly_Q